In [ ]:
# from langchain.agents import tool

# @tool
# def calculate_residency_duration(entry_date: str, exit_date: str):
#     ...

# @tool
# def get_visa_info(country: str, visa_type: str):
#     ...

# @tool
# def translate_to_lithuanian(text: str):
#     ...


In [ ]:
# ingest.py
import os
import glob
import streamlit as st
from typing import List
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from openai import OpenAI
DATA_DIR = ""
VECTOR_DIR = st.secrets.get("VECTOR_DIR", "vectorstore")


def load_docs() -> List:
    docs = []
    for path in glob.glob(os.path.join(DATA_DIR, "**/*"), recursive=True):
        if path.lower().endswith(".pdf"):
            docs.extend(PyPDFLoader(path).load())
        elif path.lower().endswith(".txt") or path.lower().endswith(".md"):
            docs.extend(TextLoader(path, encoding="utf-8").load())
    if not docs:
        raise RuntimeError("No documents found in ./data. Add PDFs or txt/md files.")
    return docs


def main():
    # os.makedirs(VECTOR_DIR, exist_ok=True)
    docs = load_docs()
    #Chunking
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=150)
    chunks = splitter.split_documents(docs)
    client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"], base_url="https://api.together.xyz/v1")
    print(chunks[0])

# Embeddings via Together AI (OpenAI-compatible)
    
    embeddings = OpenAIEmbeddings(
        # model="BAAI/bge-base-en-v1.5",
        model="togethercomputer/m2-bert-80M-32k-retrieval",
        base_url="https://api.together.xyz/v1",
        api_key=st.secrets["OPENAI_API_KEY"],
    )
    
    for i, d in enumerate(chunks):
        try:
            # embeddings.embed_query(d.page_content[:2000]) 
            print(client.embeddings.create(model="BAAI/bge-base-en-v1.5", input=d.page_content[:2000])) # test short embed
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f"❌ Chunk {i} failed, content snippet:\n{d.page_content[:2000]}")

    # Build FAISS (cosine via internal normalize)
    vs = FAISS.from_documents(chunks, embeddings)
    vs.save_local(VECTOR_DIR)
    print(f"✅ Vector store saved to {VECTOR_DIR} with {len(chunks)} chunks.")

if __name__ == "__main__":
    main()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x71cf81e46e00>>
Traceback (most recent call last):
  File "/home/helloworld/outetts/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


StreamlitSecretNotFoundError: No secrets found. Valid paths for a secrets.toml file or secret directories are: /root/.streamlit/secrets.toml, /home/helloworld/tc_02/.streamlit/secrets.toml